# Better Way 44. copyreg로 pickle을 신뢰할 수 있게 만들자

**pickle: 파이썬 객체를 바이트 스트림으로 직렬화/바이트 스트림에서 역직렬화하는 데 사용**
- 그러나 신뢰할 수 없는 부분과 통신하는 데 사용하면 안되며,
- pickle의 목적은 바이너리 채널을 통해 **프로그램 간에 파이썬 객체를 넘겨주는** 데 있음.

**Note: pickle 모듈과 json 모듈의 차이점과 사용**
- pickle 모듈로 직렬화한 데이터는 원래의 파이썬 객체를 재구성하는 데 필요한 프로그램을 담으므로, 설계 관점에서 안전하지 않음.
- 한편 json 모듈은 객체 계층에 대한 간단한 설명을 포함하므로, 역직렬화한다고 해서 파이썬 프로그램이 추가적인 위협에 노출되지는 않음.

In [1]:
class GameState(object):
    def __init__(self):
        self.level = 0
        self.lives = 4

In [2]:
# 프로그램은 게임이 실행 중일 때, 다음과 같이 레벨/생명 수 등 변화에 따라 객체를 수정함.
state = GameState()
state.level += 1
state.lives -= 1

In [3]:
# 게임이 끝나면 상태 객체를 pickle 모듈 사용하여 파일에 덤프 뜨는 게 가능함.
import pickle

state_path = '/tmp/game_state.bin'
with open(state_path, 'wb') as fp:
    pickle.dump(state, fp)

In [4]:
# 게임 다시 시작하면 파일을 로드하여 GameState 객체를 복원하는 것이 가능함.
with open(state_path, 'rb') as fp:
    state_after = pickle.load(fp)
print(state_after.__dict__)

{'level': 1, 'lives': 3}


In [5]:
class GameState(object):
    def __init__(self):
        self.level = 0
        self.lives = 4
        self.points = 0 # 새로운 필드 추가: 플레이어의 포인트 

In [6]:
state = GameState()
serialized = pickle.dumps(state) # 문자열로 직렬화
state_after = pickle.loads(serialized)
print(state_after.__dict__)

{'level': 0, 'lives': 4, 'points': 0}


In [7]:
with open(state_path, 'rb') as fp:
    state_after = pickle.load(fp)
print(state_after.__dict__)

{'level': 1, 'lives': 3}


In [8]:
assert isinstance(state_after, GameState) # old GameState 객체의 데이터임에도 불구하고, new GameState 객체라고 통과함!

위와 같은 pickle 문제는 **내장 모듈 copyreg**을 사용하면 간단하게 해결 가능함.
- copyreg 모듈로 파이썬 객체를 직렬화할 함수를 등록하여 pickle의 동작을 제어할 수 있음.
- 이에 따라 pickle을 더 신뢰할 수 있게 함.

## 기본 속성 값

GameState 객체가 언피클링 후에 항상 모든 속성을 담음을 보장하는 가장 간단한 방법은 기본 생성자를 사용하는 것이다.

In [9]:
class GameState(object):
    def __init__(self, level=0, lives=4, points=0):
        self.level = level
        self.lives = lives
        self.points = points

In [10]:
def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    return unpickle_game_state, (kwargs,)

def unpickle_game_state(kwargs):
    return GameState(**kwargs)

In [11]:
import copyreg

copyreg.pickle(GameState, pickle_game_state)

In [12]:
state = GameState()
state.points += 1000
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)
print(state_after.__dict__)

{'level': 0, 'lives': 4, 'points': 1000}


In [13]:
class GameState(object):
    def __init__(self, level=0, lives=4, points=0, magic=5):
        self.level = level
        self.lives = lives
        self.points = points
        self.magic = magic # 새로운 필드 추가

In [14]:
state_after = pickle.loads(serialized)
print(state_after.__dict__) # unpickle 함수가 GameState 생성자를 직접 호출하므로 새로운 필드도 기본값으로 추가됨

{'level': 0, 'lives': 4, 'points': 1000, 'magic': 5}


## 클래스 버전 관리

필드를 제거하여 파이썬 객체가 하위 호환성을 유지하지 않게 해야 할 때도 있다. 이 경우 기본 인수를 사용한 직렬화가 동작하지 않는다.

In [15]:
class GameState(object):
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
#        self.lives = lives
        self.points = points
        self.magic = magic

In [16]:
pickle.loads(serialized)

TypeError: __init__() got an unexpected keyword argument 'lives'

In [17]:
def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    kwargs['version'] = 2 # 버전 파라미터를 추가
    return unpickle_game_state, (kwargs,)

def unpickle_game_state(kwargs):
    version = kwargs.pop('version', 1)
    if version == 1: # 이전 버전에는 인수가 없으므로 이에 맞춰서 GameState 생성자에 넘길 인수를 조작
        kwargs.pop('lives')
    return GameState(**kwargs)

In [18]:
copyreg.pickle(GameState, pickle_game_state)
state_after = pickle.loads(serialized)
print(state_after.__dict__)

{'level': 0, 'points': 1000, 'magic': 5}


## 안정적인 임포트 경로

클래스의 이름을 변경하면 pickle 모듈이 제대로 동작하지 않을 수 있다.

In [21]:
copyreg.dispatch_table.clear() 
state = GameState()
serialized = pickle.dumps(state)
del GameState

class BetterGameState(object):
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
        self.points = points
        self.magic = magic

In [23]:
pickle.loads(serialized)

AttributeError: Can't get attribute 'GameState' on <module '__main__'>

In [24]:
print(serialized[:25]) # 직렬화한 객체의 클래스를 임포트할 경로가 피클 데이터에 인코드되어 있음.

b'\x80\x03c__main__\nGameState\nq\x00)'


In [26]:
copyreg.pickle(BetterGameState, pickle_game_state) # 다시 copyreg 사용, 언피클하는 데 사용할 함수의 안정적인 식별자를 설정
state = BetterGameState()
serialized = pickle.dumps(state)

In [28]:
print(serialized[:35]) # pickle_game_state의 임포트 경로가 인코드되어 있음.

b'\x80\x03c__main__\nunpickle_game_state\nq\x00}'


문제점: unpickle_game_state 함수가 존재하는 모듈의 경로를 변경할 수 없음. 
- 어떤 함수로 데이터를 직렬화한 후에는 나중에 역직렬화 할 때 사용할 수 있도록 해당 임포트 경로에 함수를 남겨둬야 함.

**핵심 정리**
- 내장 모듈 pickle은 신뢰할 수 있는 프로그램 간에 객체를 직렬화/역직렬화하는 용도로만 사용할 수 있다.
- pickle 모듈은 간단한 사용 사례를 벗어나는 용도로 사용하면 제대로 동작하지 않을 수도 있다.
- 빠뜨린 속성 값을 추가하거나 클래스에 버전 관리 기능을 제공하거나 안정적인 임포트 경로를 제공하려면 pickel과 함께 내장 모듈 copyreg을 사용해야 한다.